In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from GridWorld import Task

In [ ]:
#
grid_world_size = (3, 3)
n_abstract_actions=4
list_walls = []

list_start_locations = [[]]